In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io, filters
import numpy as np

from tqdm.notebook import tqdm

from PIL import Image
import numpy as np

import cv2
import copy
import time

from mpire import WorkerPool

from PIL import Image
from matplotlib import cm
from tqdm.notebook import tqdm


from combra import stats as cstats
from combra import approx as capprox
from combra import image as cimage
from combra import contours as ccontours

from numba import njit
import numpy as np
import cv2
import os

from combra.contours import contour_to_binary_mask, scale_contour, draw_contours

import json
from pathlib import Path
import re

import pandas as pd

import pyarrow as pa
import pyarrow.parquet as pq

from combra import data as cdata 

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.colors as pc


# Generate data

In [ ]:
poliamid_path = "data/autumn/images"

output_path = "poliamid_group_250_test.parquet"

poliamid_dataset = cdata.PoliamidDataset(poliamid_path)


In [ ]:
poliamid_dataset.generate(n_jobs=23, out_path=output_path)

In [ ]:
scale

In [ ]:
img_path = poliamid_dataset.image_groups[0][1][100]

image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
image = image[400:-400,1300:-1300]

_, cnts = poliamid_dataset.preprocess_image(image)

# 56 
# 11
contour = cnts[11]

binary = contour_to_binary_mask(contour, thickness=1)

print(len(contour))
plt.imshow(binary, cmap='gray')
plt.savefig('binary.jpg', bbox_inches='tight')
plt.show()

sizes, scale = cimage.valid_box_sizes_from_shape(binary.shape)

cnt_big = ccontours.scale_contour(contour, scale)

binary_big = ccontours.contour_to_binary_mask(cnt_big, thickness=1)

plt.imshow(binary_big, cmap='gray')
plt.savefig('binary_big.jpg', bbox_inches='tight')
plt.show()

D = cimage.image_fractal_dimension(binary_big, sizes)

# print(D)


In [ ]:
missing = 4
min_hw = 8

# scale must be power of 2 to preserve box-counting grid
scale = 1 << missing

# recompute sizes in scaled space
max_size_scaled = (min_hw * scale) // 2
max_exp_scaled = int(np.log2(max_size_scaled))

exps_scaled = np.arange(1, max_exp_scaled + 1)

(1 << exps_scaled), scale

# Images fractal dimention

In [ ]:
# Use Parquet generated by the fractal-dimension extraction cell
in_path = Path("poliamid_group_250_test.parquet")

table = pq.read_table(in_path)
data = table.to_pydict()


In [ ]:
g = groups[15]

fd_list = g.get("fd_image_list", [])

print(step)
print(fd_list)

print('stats',cstats.stats_preprocess(fd_list, step))

plt.hist(fd_list,bins=100)
plt.show()

In [ ]:
groups.__len__()

In [ ]:

n = len(data["group_label"])
groups = [{k: data[k][i] for k in data.keys()} for i in range(n)]
group_labels = [g["group_label"] for g in groups]

# bin size for stats_preprocess

nrows, ncols = 6, 3
page_size = nrows * ncols

for step in tqdm([0.0005]):

    # Colors per group (will cycle if many groups)
    colors = plt.cm.tab20(np.linspace(0, 1, max(2, min(20, len(groups)))))

    # If there are more than 18 groups, page across multiple 3x6 figures
    for page_start in range(0, len(groups), page_size):
        page_groups = groups[page_start : page_start + page_size]

        fig, axes = plt.subplots(
            nrows,
            ncols,
            figsize=(ncols * 4, nrows * 3),
            sharex=True,
            sharey=True,
        )
        axes = np.asarray(axes).ravel()

        # Hide unused axes on the last page
        for ax in axes[len(page_groups) :]:
            ax.axis("off")

        for i, g in enumerate(page_groups):
            ax = axes[i]
            idx = page_start + i

            fd_list = g.get("fd_image_list", [])
            if fd_list is None:
                fd_list = []
            else:
                fd_list = list(fd_list)

            color = colors[idx % len(colors)]

            ax.set_xlim(1, 2)
            ax.set_ylim(0, 0.12)
            ax.grid(True, alpha=0.2)

            if not fd_list:
                ax.set_title(f"{g.get('group_label', '')}\n(no data)", fontsize=9)
                continue

            x_orig, y_orig = cstats.stats_preprocess(fd_list, step)

            (x_fit, y_fit), mus, sigmas, amps = capprox.bimodal_gauss_approx(
                x_orig,
                y_orig,
                mu1=1.2,
                mu2=1.8,
                sigma1=0.1,
                sigma2=0.1,
                amp1=1,
                amp2=1,
            )

            ax.plot(x_orig, y_orig, "-o", color=color, markersize=3, linewidth=1)
            ax.plot(x_fit, y_fit, "--", color=color, linewidth=1, alpha=0.8)

            ax.set_title(
                f"{g['group_label']}\n"
                f"mu1={mus[0]:.2f} mu2={mus[1]:.2f}  s1={sigmas[0]:.2f} s2={sigmas[1]:.2f}",
                fontsize=8,
            )

        fig.suptitle(
            f"Fractal dimension distributions (step={step})\n"
            f"groups {page_start + 1}-{page_start + len(page_groups)} of {len(groups)}",
            fontsize=14,
        )
        fig.supxlabel("fractal dimension", fontsize=12)
        fig.supylabel("p(x)", fontsize=12)

        plt.tight_layout()

        plt.savefig(f'fractal_dimention_step={step}_page={page_start // page_size + 1}.jpg', bbox_inches='tight')
        plt.show()


# cnts fractal dimention

In [ ]:

n = len(data["group_label"])
groups = [{k: data[k][i] for k in data.keys()} for i in range(n)]
group_labels = [g["group_label"] for g in groups]

# bin size for stats_preprocess
for step in tqdm([0.01, 0.05, 0.1, 0.2, 0.3]):
# for step in tqdm([0.001, 0.005, 0.01, 0.05, 0.1]):

    fig, axes = plt.subplots(1, 2, figsize=(20, 7))

    # Colors per group (will cycle if many groups)
    colors = plt.cm.tab20(np.linspace(0, 1, max(2, min(20, len(groups)))))

    a_list = []

    for idx, g in enumerate(groups):
        # fd_list = g.get("fd_list", [])
        fd_list = g.get("fd_image_list", [])
        if fd_list is None:
            fd_list = []
        else:
            fd_list = list(fd_list)

        color = colors[idx % len(colors)]

        if not fd_list:
            a_list.append(np.nan)
            continue

        x_orig, y_orig = cstats.stats_preprocess(fd_list, step)

        # Exponential distribution only
        (x_fit, y_fit), a, amp_exp = capprox.exponential_approx(
            x_orig, y_orig, a=1, amp=1, x_lim=[1, 2], N=20
        )
        a_list.append(a)

        label = (
            # f"{g['group_label']} | imgs={g.get('n_images', 0)} | cnts={g.get('n_contours', 0)} | a={a:.4f}"
            f"{g['group_label']} | a={a:.4f}"
        )

        # axes[0].plot(x_fit, y_fit, '--', linewidth=2, color=color)
        # axes[0].plot(x_orig, y_orig, '-o', color=color, label=label)
        axes[0].scatter(x_orig, y_orig, color=color, label=label)

    axes[0].set_title(f'exponential fit, step={step}', fontsize=15)
    axes[0].set_xlim(1, 2)
    # axes[0].set_ylim(1e-8, 1)
    axes[0].set_yscale('log')
    axes[0].set_ylabel('p(x)', fontsize=15)
    axes[0].set_xlabel('fractal dimension', fontsize=15)
    axes[0].legend(loc='upper right', fontsize=10)

    # Diagnostics: parameter a per group
    axes[1].plot(range(len(groups)), a_list, '-o', label='a')
    axes[1].set_title(f'exponential parameter a, step={step}', fontsize=15)
    axes[1].set_xticks(range(len(groups)))
    axes[1].set_xticklabels(group_labels, rotation=45, fontsize=12, ha='right')
    axes[1].legend(loc='upper right', fontsize=12)

    # plt.savefig(f'fractal_dimention_step={step}.jpg', bbox_inches='tight')

    plt.tight_layout()
    plt.show()
